In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime

In [20]:
# Logging function
def log_progress(message):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open("code_log.txt", "a") as log_file:
        log_file.write(f"{timestamp} : {message}\n")

In [21]:
# Extraction function
def extract(url, table_attribs):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = pd.read_html(str(soup.find_all('table')))

    print(f"Total de tabelas extraídas: {len(tables)}")
    for idx, table in enumerate(tables):
        print(f"\nTabela {idx} - Colunas: {table.columns}")
        print(table.head())

    for table in tables:
        if set(table_attribs).issubset(table.columns):
            df = table[table_attribs]
            df = df.rename(columns={"Bank name": "Name", "Market cap (US$ billion)": "MC_USD_Billion"})
            df["MC_USD_Billion"] = df["MC_USD_Billion"].astype(str).str.replace('', '', regex=True).str.replace(',', '', regex=True).astype(float)
            log_progress("Data extraction complete. Initiating Transformation process")
            return df

        raise KeyError("Nenhuma tabela contendo as colunas especificadas foi encontrada.")

In [22]:
# Transformation function
def transform(df, csv_path):
    exchange_rates = pd.read_csv(csv_path, index_col=0).squeeze().to_dict()
    df['MC_GBP_Billion'] = [np.round(x * exchange_rates['GBP'], 2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x * exchange_rates['EUR'], 2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x * exchange_rates['INR'], 2) for x in df['MC_USD_Billion']]
    log_progress("Data transformation complete. Initiating Loading process")
    return df

In [23]:
# Load to CSV function
def load_to_csv(df, output_path):
    df.to_csv(output_path, index=False)
    log_progress("Data saved to CSV file")

In [24]:
# Load to database function
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
    log_progress("Data loaded to Database as a table, Executing queries")

In [25]:
# Run SQL queries function
def run_query(query_statement, sql_connection):
    cursor = sql_connection.cursor()
    cursor.execute(query_statement)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    log_progress("Process Complete")

In [26]:
# Main execution
if __name__ == "__main__":
    log_progress("Preliminaries complete. Initiating ETL process")

    url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
    csv_path = "exchange_rate.csv"
    exchange_rate_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv"
    output_csv_path = "Largest_banks_data.csv"
    db_name = "Banks.db"
    table_name = "Largest_banks"
    table_attributes = ["Bank name", "Market cap (US$ billion)"]

    df_extracted = extract(url, table_attributes)
    import os
    if not os.path.exists(csv_path):
        log_progress("Exchange rate file not found. Downloading...")
        response = requests.get(exchange_rate_url)
        with open(csv_path, 'wb') as file:
            file.write(response.content)
        log_progress("Exchange rate file downloaded successfully.")

    df_transformed = transform(df_extracted, csv_path)
    load_to_csv(df_transformed, output_csv_path)

    sql_connection = sqlite3.connect(db_name)
    log_progress("SQL Connection initiated")
    load_to_db(df_transformed, sql_connection, table_name)

    run_query("SELECT * FROM Largest_banks", sql_connection)
    run_query("SELECT AVG(MC_USD_Billion) FROM Largest_banks", sql_connection)
    run_query("SELECT Name FROM Largest_banks LIMIT 5", sql_connection)

    sql_connection.close()
    log_progress("Server Connection closed")

<ipython-input-21-0dc156273fe0>:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(soup.find_all('table')))


Total de tabelas extraídas: 3

Tabela 0 - Colunas: Index(['Rank', 'Bank name', 'Market cap (US$ billion)'], dtype='object')
   Rank                                Bank name  Market cap (US$ billion)
0     1                           JPMorgan Chase                    432.92
1     2                          Bank of America                    231.52
2     3  Industrial and Commercial Bank of China                    194.56
3     4               Agricultural Bank of China                    160.68
4     5                                HDFC Bank                    157.91

Tabela 1 - Colunas: Index(['Rank', 'Bank name', 'Total assets (2022) (US$ billion)'], dtype='object')
   Rank                                        Bank name  \
0     1  Industrial and Commercial Bank of China Limited   
1     2                          China Construction Bank   
2     3                       Agricultural Bank of China   
3     4                                    Bank of China   
4     5                